In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
project_dir = 'C:\\Users\\PLDD\\python\\Python\\ML\\Kaggle\\house_price'
raw_path = os.path.join(project_dir,'data','raw')
train_path = os.path.join(raw_path, 'train.csv')
test_path = os.path.join(raw_path, 'test.csv')

 # Create data frame

Combines train and test data frame:

In [217]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
test_df['SalePrice'] = 0.0
df = train_df.append(test_df)
df.index = list(range(train_df.index.size + test_df.index.size))
# 2916 out of 2919 Utilities are AllPub so i think it should be droped off
df = df.drop('Utilities', axis = 1)

# Proccess outliers

In [218]:
#'LotFrontage', 'LotArea', 'MasVnrArea', 'GrLivArea', 'GarageArea', 'PoolArea', 'SalePrice']
#copydf['LotFrontage'].plot.hist()
#copydf['LotFrontage'].plot.box()
#copydf['LotArea'].plot.hist()
#copydf['LotArea'].plot.box()
#copydf['GrLivArea'].plot.hist()
#copydf['GrLivArea'].plot.box()
#plt.show()
lx = (df['LotFrontage'] > 200) | (df['LotArea'] > 100000) | (df['GrLivArea'] > 4000)
#df = df.drop(df.index[lx], axis = 0)
df.loc[df['LotFrontage'] > 200,'LotFrontage'] = np.nan
df.loc[df['LotArea'] > 100000,'LotArea'] = np.nan
df.loc[df['GrLivArea'] > 4000,'GrLivArea'] = np.nan

# Proccess Nan values

In [219]:
DefSeqCat = ['Alley','BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu',
       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC',
       'Fence', 'MiscFeature']
UndefSeqCat1 = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd','Electrical', 'KitchenQual',
            'Functional', 'SaleType'] # there are defenetly lost elements 
UndefSeqCat2 = ['MasVnrType'] # i may just not make it clear the property of the column
UndefSeqNum = ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 
               'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea']

replace=dict.fromkeys(DefSeqCat, 'NoNe')
df = df.fillna(replace)
#'MasVnrType'
ix = df.index[df['MasVnrType'].isnull()]
df.loc[ix,'MasVnrType'] = 'NoNe'
df.loc[ix,'MasVnrArea'] = 0

NanCol = ['LotFrontage', 'MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual',
       'Functional', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'SaleType']

Replace NAN values in 'LotFrontage' and 'LotArea' with iterpolated value.

In [220]:
def ReplaceNanValues_LotFrontageArea(df, alables, abins, fromfld, tofld, catclass, targetclass):
    fullclass = catclass + [targetclass]
    df[tofld]=pd.cut(x = df[fromfld].values, bins = abins, right = False, labels = alables)
    validdf = df[fullclass].dropna(axis = 0, how = 'any')
    nandf = df[df[targetclass].isnull()]
    pvt = validdf.pivot_table(values = targetclass, index = catclass, aggfunc = np.mean)
    pvt=pvt[pvt.notnull().all(1)]
    t1 = pvt.loc[[tuple(x) for x in nandf[catclass].values]]
    t1.index = nandf.index
    df.loc[nandf.index,targetclass] = t1[targetclass]
    df = df.drop(tofld, axis = 1)
    return df

In [221]:
copydf = df.copy()
yearlable = ['1906', '1942', '1975', '1991', '2011']
yearbin = [1800,1906,1942,1975, 1991, 2011]

#case 1
catclass = ['Neighborhood', 'LotConfig','LotShape','YearInt']
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotFrontage' )
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotArea')
#print(copydf[copydf['LotFrontage'].isnull()].index.size)

catclass = ['LotConfig','LotShape','YearInt']
#case 2
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotFrontage')
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotArea')
#print(copydf[copydf['LotFrontage'].isnull()].index.size)

#case 3
yearlable = ['1942', '1991', '2011']
yearbin = [1800, 1942, 1991, 2011]
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotFrontage')
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotArea')
#print(copydf[copydf['LotFrontage'].isnull()].index.size)

#case 4
yearlable = ['1975', '1991', '2011']
yearbin = [1800, 1975, 1991, 2011]
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotFrontage')
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotArea')
#print(copydf[copydf['LotFrontage'].isnull()].index.size)

#case 5
yearlable = ['2011']
yearbin = [1800, 2011]
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotFrontage')
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotArea')
#print(copydf[copydf['LotFrontage'].isnull()].index.size)
#print(copydf[copydf['LotArea'].isnull()].index.size)
df = copydf

83
0
0


Replace Nan in 'MSZoning'

In [222]:
ind1 = [1915, 2216, 2250] # C - assign by many factors
ind2 = [2904] # RL - by Lot Area, LotFrontage and YearBuilt 
df.loc[ind1,'MSZoning'] = 'C'
df.loc[ind2,'MSZoning'] = 'RL'

Replace Nan in 'Exterior1st' and 'Exterior2nd'

In [223]:
targetclass = ['Exterior1st', 'Exterior2nd']
ind = [2151]
df.loc[ind,targetclass] = 'AsbShng'

Add column 'RangeYrBlt' and remove columns 'GarageYrBlt' and 'YearBilt'

In [233]:
yearbins = [1800, 1895, 1905, 1917, 1927, 1937,1947,1960, 1973, 1989, 1998, 2012]
yearlabels = ['1895', '1905', '1917', '1927', '1937', '1947', '1960', '1973', '1989', '1998', '2012']
if df.columns.isin(['YearBuilt']).any():
    df['RangeYrBlt']=pd.cut(x = df['YearBuilt'].values, bins = yearbins, right = False, labels = yearlabels)
    df = df.drop('YearBuilt', axis = 1)
if df.columns.isin(['GarageYrBlt']).any():
    df = df.drop('GarageYrBlt', axis = 1)

Replace Nan in 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF' (row index - 2120)

In [252]:
# There is no basement in this house so all areas we take as 0
df.loc[2120,['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF']] = 0

Replase Nan in 'Electrical' (row index - 1379)

In [261]:
df.loc[1379, 'Electrical'] = 'SBrkr' #  all houses in 2012 year built have this type of Electrical

Replase Nan in 'GrLivArea'

In [277]:
if df.columns.isin(['GrLivArea']).any():
    df = df.drop('GrLivArea', axis = 1) # it is sum of '1stFlrSF' and '2ndFlrSF'

Replace Nan in 'BsmtFullBath' and 'BsmtHalfBath' (row indexes are 2120 and 2188) # all these houses have no basements

In [297]:
df.loc[[2120,2188],['BsmtFullBath', 'BsmtHalfBath']] = 0

Replace Nan in 'KitchenQual' (row index is 1555)

In [311]:
# Value of 'KitchenQual' depends on 'OverallQual', 'OverallCond'
df.loc[1555, 'KitchenQual'] = 'TA'

Replace Nan in 'Functional' (row indexes are 2216, 2473)

In [323]:
# The most used functionality is 'Typ'
df.loc[[2216, 2473],'Functional'] = 'Typ'

Replace Nan in 'GarageCars', 'GarageArea' (row index is 2576)

In [333]:
#there is no garage so its area is 0
df.loc[2576,'GarageArea'] = 0
if df.columns.isin(['GarageCars']).any():
    df = df.drop('GarageCars', axis = 1) # it has straight dependency on garage area

Replace Nan in 'SaleType' (row index is 2489)

In [349]:
df.loc[2489, 'SaleType'] = 'WD' # the most used type according to sold year and sale condition

# Categorizing

In [411]:
#copydf = df.copy()
copydf.columns[copydf.dtypes == 'object']

Index(['Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir',
       'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC',
       'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'],
      dtype='object')

In [430]:
OrderedCat = ['MSZoning', 'Street','Alley', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofMatl']
NonOrderedCat = ['LandContour', 'LotConfig', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'RoofStyle'
                'Exterior1st','Exterior2nd']

copydf['MSZoning'] = pd.Categorical(copydf['MSZoning'].values, ['C', 'C (all)', 'RH', 'FV', 'RM', 'RL'], ordered = True)
copydf['Street'] = pd.Categorical(copydf['Street'].values, ['Grvl','Pave'], ordered = True)
copydf['Alley'] = pd.Categorical(copydf['Alley'].values, ['NoNe', 'Grvl', 'Pave'], ordered = True)
copydf['LotShape'] = pd.Categorical(copydf['LotShape'].values, ['Reg', 'IR1', 'IR2', 'IR3'], ordered = False)
copydf['LandContour'] = pd.Categorical(copydf['LandContour'].values, ['Lvl', 'Bnk', 'Low', 'HLS'], ordered = False)
copydf['LotConfig'] = pd.Categorical(copydf['LotConfig'].values, ['Inside', 'FR2', 'Corner', 'CulDSac', 'FR3'], ordered = False)
copydf['LandSlope'] = pd.Categorical(copydf['LandSlope'].values, ['Gtl', 'Mod', 'Sev'], ordered = False)
copydf['Neighborhood'] = pd.Categorical(copydf['Neighborhood'].values, ['CollgCr', 'Veenker', 'Crawfor', 'NoRidge', 'Mitchel', 'Somerst',
       'NWAmes', 'OldTown', 'BrkSide', 'Sawyer', 'NridgHt', 'NAmes',
       'SawyerW', 'IDOTRR', 'MeadowV', 'Edwards', 'Timber', 'Gilbert',
       'StoneBr', 'ClearCr', 'NPkVill', 'Blmngtn', 'BrDale', 'SWISU',
       'Blueste'], ordered = False)
copydf['Condition1'] = pd.Categorical(copydf['Condition1'].values, ['PosA', 'PosN', 'Norm', 'RRNn', 'RRNe', 'RRAe',  'RRAn', 'Feedr', 'Artery'], ordered = True)
copydf['Condition2'] = pd.Categorical(copydf['Condition2'].values, ['PosA', 'PosN', 'Norm', 'RRNn', 'RRAe', 'RRAn','Feedr', 'Artery'], ordered = True)
copydf['BldgType'] = pd.Categorical(copydf['BldgType'].values, ['2fmCon', 'Duplex', 'Twnhs', 'TwnhsE', '1Fam'], ordered = True)
copydf['HouseStyle'] = pd.Categorical(copydf['HouseStyle'].values, ['1.5Unf', '1.5Fin', 'SFoyer', 'SLvl', '1Story', '2.5Unf', '2.5Fin', '2Story'], ordered = True)
copydf['RoofStyle'] = pd.Categorical(copydf['RoofStyle'].values, ['Gable', 'Hip', 'Gambrel', 'Mansard', 'Flat', 'Shed'], ordered = False)
copydf['RoofMatl'] = pd.Categorical(copydf['RoofMatl'].values, ['Roll', 'ClyTile', 'Metal', 'CompShg', 'Tar&Grv', 'Membran', 'WdShake', 'WdShngl'], ordered = True)
copydf['Exterior1st'] = pd.Categorical(copydf['Exterior1st'].values, ['VinylSd', 'MetalSd', 'Wd Sdng', 'HdBoard', 'BrkFace', 'WdShing', 'CemntBd', 'Plywood', 'AsbShng', 'Stucco', 'BrkComm', 'AsphShn','Stone', 'ImStucc', 'CBlock'], ordered = False)
copydf['Exterior2nd'] = pd.Categorical(copydf['Exterior2nd'].values, ['VinylSd', 'MetalSd', 'Wd Shng', 'HdBoard', 'Plywood', 'Wd Sdng','CmentBd', 'BrkFace', 'Stucco', 'AsbShng', 'Brk Cmn', 'ImStucc','AsphShn', 'Stone', 'Other', 'CBlock'], ordered = False)




In [432]:
fld = 'Exterior2nd'
copydf[fld].unique()

array(['VinylSd', 'MetalSd', 'Wd Shng', 'HdBoard', 'Plywood', 'Wd Sdng',
       'CmentBd', 'BrkFace', 'Stucco', 'AsbShng', 'Brk Cmn', 'ImStucc',
       'AsphShn', 'Stone', 'Other', 'CBlock'], dtype=object)

In [433]:
pvt=train_df.pivot_table(values = 'SalePrice', index = [fld], aggfunc = [np.mean, np.std, np.size, np.max])
pvt

,mean,std,size,amax
,SalePrice,SalePrice,SalePrice,SalePrice
Exterior2nd,,,,
AsbShng,114060.550000,42315.043608,20,225000
AsphShn,138000.000000,37509.998667,3,175000
Brk Cmn,126714.285714,38693.084161,7,155000
BrkFace,195818.000000,95097.551756,25,430000
CBlock,105000.000000,NaN,1,105000
CmentBd,230093.833333,116140.396221,60,556581
HdBoard,167661.565217,70061.078357,207,755000
ImStucc,252070.000000,193176.672447,10,745000


In [ ]:
copydf['MSZoning'] = CategoricalDtype(['FV', ''])